# Pipeline walkthrough

## Data import

In [1]:
from preprocessing.import_data import import_save_dataset

import napari

trim_series = True
lif_test_name = "test_data/2021-06-14/p2pdpwt"
lsm_test_name = "test_data/2023-04-07/p2pdp_zld-sites-ctrl_fwd_1"
var1_test_name = "test_data/2023-06-16/RBSPWM_embryo01"

(
    channels_full_dataset,
    original_global_metadata,
    original_frame_metadata,
    export_global_metadata,
    export_frame_metadata,
) = import_save_dataset(var1_test_name, trim_series=trim_series, mode="tiff")

/home/yovan/mambaforge/envs/transcription_pipeline/lib/python3.10/site-packages/pims/bioformats.py:410: UserWarning: Due to an issue with JPype 0.6.0, reading is slower. Please consider upgrading JPype to 0.6.1 or later.
  warn('Due to an issue with JPype 0.6.0, reading is slower. '


The series in ['Series001', 'Series002', 'Series003'] have inconsistent ChannelPinholeSize, check your imaging settings and metadata.
The series in ['Series001', 'Series002', 'Series003'] have inconsistent LaserID, check your imaging settings and metadata.


/home/yovan/Documents/Berkeley/github_repositories/transcription_pipeline/preprocessing/import_data.py:719: UserWarning: /home/yovan/Documents/Berkeley/github_repositories/transcription_pipeline/test_data/2023-06-16/RBSPWM_embryo01/collated_dataset/collated_dataset_ch00.tiff is a low contrast image
  imsave(collated_data_path, channel_data, plugin="tifffile")
/home/yovan/Documents/Berkeley/github_repositories/transcription_pipeline/preprocessing/import_data.py:719: UserWarning: /home/yovan/Documents/Berkeley/github_repositories/transcription_pipeline/test_data/2023-06-16/RBSPWM_embryo01/collated_dataset/collated_dataset_ch01.tiff is a low contrast image
  imsave(collated_data_path, channel_data, plugin="tifffile")


In [2]:
nuclear_channel_metadata = export_frame_metadata[1]
nuclear_channel = channels_full_dataset[1]

## Starting a Dask Client for parallelization

In [3]:
from dask.distributed import LocalCluster, Client

In [4]:
cluster = LocalCluster(
    host="localhost",
    scheduler_port=8786,
    threads_per_worker=1,
    n_workers=12,
    memory_limit="4GB",
)

In [5]:
client = Client(cluster)

In [6]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 12,Total memory: 44.70 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:8786,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 44.70 GiB
Comm: tcp://127.0.0.1:44009,Total threads: 1
Dashboard: http://127.0.0.1:44509/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:44375,


## Nuclear segmentation and tracking

In [7]:
import pipeline

In [8]:
%%time

nuclear_tracking = pipeline.Nuclear(
    data=nuclear_channel,
    global_metadata=export_global_metadata[1],
    frame_metadata=nuclear_channel_metadata,
    client=client,
)

nuclear_tracking.track_nuclei()

Frame 36: 320 trajectories present.


SubnetOversizeException: search_range (aka maxdisp) too large for reasonable performance on these data (sub net contains 131 points)

In [12]:
nuclear_segmentation = nuclear_tracking.labels_futures

In [14]:
import numpy as np

In [15]:
nuclear_segmentation = np.concatenate(client.gather(nuclear_segmentation))

In [9]:
nuclear_labels_futures = nuclear_tracking.reordered_labels_futures

AttributeError: 'Nuclear' object has no attribute 'reordered_labels_futures'

In [ ]:
test_mask = client.gather(nuclear_tracking.mask_futures)

In [ ]:
import numpy as np

In [ ]:
test_mask = np.concatenate(test_mask)

In [16]:
viewer = napari.view_image(nuclear_segmentation)
napari.run()

In [17]:
viewer.add_labels(nuclear_segmentation)

<Labels layer 'nuclear_segmentation' at 0x7f9174253df0>

In [18]:
from skimage.filters import difference_of_gaussians

In [19]:
nuclear_tracking.default_params

{'denoise_params': {'denoising': 'gaussian',
  'denoising_sigma': array([1.2623927, 2.       , 2.       ])},
 'binarize_params': {'thresholding': 'global_otsu',
  'closing_footprint': array([[[False,  True, False],
          [ True,  True,  True],
          [False,  True, False]],
  
         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]],
  
         [[False,  True, False],
          [ True,  True,  True],
          [False,  True, False]]])},
 'mark_params': {'low_sigma': array([5.17057978, 6.33376631, 6.33376504]),
  'high_sigma': array([25.8528989 , 31.66883155, 31.6688252 ]),
  'max_footprint': ((1, 24),
   array([[[False,  True, False],
           [ True,  True,  True],
           [False,  True, False]],
   
          [[ True,  True,  True],
           [ True,  True,  True],
           [ True,  True,  True]],
   
          [[False,  True, False],
           [ True,  True,  True],
           [False,  True, False]]])),
  'max_diff': 1},
 's

In [28]:
low_sigma = np.array([5.17057978, 6.33376631, 6.33376504])
high_sigma = np.array([10.8528989 , 15.66883155, 15.6688252 ])

In [29]:
dog_movie = np.concatenate([difference_of_gaussians(frame, low_sigma=low_sigma, high_sigma=high_sigma) for frame in nuclear_channel])

In [30]:
dog_movie = np.reshape(dog_movie, nuclear_channel.shape)

In [31]:
nuclear_channel.shape

(74, 21, 256, 1024)

In [32]:
viewer.add_image(dog_movie)

<Image layer 'dog_movie [1]' at 0x7f9183283790>

## Spot segmentation and fitting

It can be shown for an xy-symmetric Gaussian that
$$
\int_{\mathbb{R}^3} A e^{- \frac{x^2 + y^2}{2 \sigma_{xy}^2} - \frac{z^2}{2 \sigma_z^2}} \ dx \ dy \ dz = 2 \sqrt{2} A \pi^{3/2} \sigma_{xy}^2 \sigma_z
$$
where we have used notation consistent with that used in `spot_analysis.fitting`. We can therefore use simple algebraic manipulation of the fit parameters to estimate the spot intensities.

In [ ]:
transcription_channel_metadata = export_frame_metadata[0]
transcription_channel = channels_full_dataset[0]

In [ ]:
%%time

spot_tracking = pipeline.Spot(
    data=transcription_channel,
    global_metadata=export_global_metadata[0],
    frame_metadata=transcription_channel_metadata,
    nuclear_labels=nuclear_labels_futures,
    client=client,
    keep_bandpass=False,
)

spot_tracking.extract_spot_traces()

In [ ]:
spot_labels = spot_tracking.reordered_spot_labels

## Compiling traces

In [ ]:
from spot_analysis import compile_data

In [ ]:
spot_dataframe = spot_tracking.spot_dataframe
mitosis_dataframe = nuclear_tracking.mitosis_dataframe

compiled_dataframe = compile_data.compile_traces(
    spot_dataframe,
    nuclear_tracking_dataframe=mitosis_dataframe,
    compile_columns_nuclear=["nuclear_cycle", "division_time"],
)

In [ ]:
compiled_dataframe

## Visualizing traces

In [ ]:
from utils import plottable

In [ ]:
%matplotlib widget
# This is taken from https://stackoverflow.com/questions/18390461/scroll-backwards-and-forwards-through-matplotlib-plots
import matplotlib.pyplot as plt

traces = plottable.generate_trace_plot_list(compiled_dataframe)

curr_pos = 0


def key_event(e):
    global curr_pos

    if e.key == "right":
        curr_pos = curr_pos + 1
    elif e.key == "left":
        curr_pos = curr_pos - 1
    else:
        return
    curr_pos = curr_pos % len(traces)

    ax.cla()
    ax.plot(traces[curr_pos][0], traces[curr_pos][1], ".")
    ax.set_xlabel("time (s)")
    ax.set_ylabel("Spot intensity (AU)")
    ax.set_title(f"Particle {traces[curr_pos][2]}, NC {traces[curr_pos][3]}")
    fig.canvas.draw()


fig = plt.figure()
fig.canvas.mpl_connect("key_press_event", key_event)

ax = fig.add_subplot(111)
ax.plot(traces[curr_pos][0], traces[curr_pos][1], ".")
ax.set_xlabel("time (s)")
ax.set_ylabel("Spot intensity (AU)")
ax.set_title(f"Particle {traces[curr_pos][2]}, NC {traces[curr_pos][3]}")

plt.show()

In [ ]:
from tracking import detect_mitosis

In [ ]:
viewer = napari.view_image(nuclear_channel, name="Nuclear Channel")
viewer.add_labels(nuclear_labels)
_ = detect_mitosis.tracks_to_napari(
    viewer, mitosis_dataframe, name="nuclear_tracks", output=False
)
viewer.add_image(transcription_channel, name="Transcription Channel")
viewer.add_labels(spot_labels)
napari.run()